# 4.7: Deriving New Variables Task Work

## This script includes the following points:

### Q1a.  Create "price_label" column.

### Q1b.  Create "busiest_day" column.

### Q2.  Update "busiest day" to "busiest days" which will show the order day of week as "Busiest days" "Slowest days" or "Regularly busy."

### Q3.  Check the values of this new column for accuracy.  Note any observations in markdown format.

### Q4.  Create new column "busiest_period_of_day" to show periods labeled "Most orders", "Average orders" and "Fewest orders."

### Q5.  Print the frequency for this column.

### Q7.  Export dataframe as a pickle file to "Prepared Data" folder.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create path
path = r'/Users/andrewruhnow/Documents/12-2023 Instacart Basket Analysis'

In [3]:
path

'/Users/andrewruhnow/Documents/12-2023 Instacart Basket Analysis'

In [4]:
# Import orders_products_merged.pkl file
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

# Q1a.  Create "price_label" column.

In [5]:
# Creat subset of first million rows
df = df_ords_prods_merge[:1000000]

In [6]:
df.shape

(1000000, 15)

In [7]:
# Define a function
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'

In [8]:
# Apply price range function.
df['price_range'] = df.apply(price_label, axis=1)

/var/folders/qg/tbv61t9j3ngdmw6m2qtbk34w0000gn/T/ipykernel_7829/3431473277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis=1)


In [9]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    659373
Low-range product    331283
High range             9344
Name: count, dtype: int64

In [10]:
df['prices'].max()

24.5

In [11]:
# Using loc() method on df subset
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/qg/tbv61t9j3ngdmw6m2qtbk34w0000gn/T/ipykernel_7829/3711773602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [12]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [13]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [14]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     659373
Low-range product     331283
High-range product      9344
Name: count, dtype: int64

In [15]:
# Use loc() method on ords_prods_merge dataframe
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [16]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [17]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [18]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21890146
Low-range product     10126384
High-range product      417682
Name: count, dtype: int64

# Q1b.  Create "busiest_day" column.

In [22]:
# Checking that column name "orders_dow" is still renamed "orders_day_of_week"
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,add_to_cart_order,reordered,_merge,price_range_loc
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product


In [23]:
# Find frequency of orders_day_of_week.
df_ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6209632
1    5665830
6    4500246
2    4217766
5    4209449
3    3844096
4    3787193
Name: count, dtype: int64

In [25]:
# Use for-loop to determine whether orders are on "busiest day" (Saturday), "least busy" (Wednesday), or "regularly busy" (other days)
result = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [26]:
result

['Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Busiest day',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Regularly busy',
 'Busiest day',
 'Regularly busy',
 'Busiest day',
 'Least busy',
 'Busiest day',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Bus

In [27]:
# Create new column in ords_prods_merge to view results
df_ords_prods_merge['busiest_day'] = result

In [28]:
df_ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22437387
Busiest day        6209632
Least busy         3787193
Name: count, dtype: int64

# Q2.  Update "busiest day" to "busiest days" which will show the order day of week as "Busiest days" "Slowest days" or "Regularly busy."

In [29]:
# Create column "busiest_days" from "orders_day_of_week"

result_1 = []

for value in df_ords_prods_merge["orders_day_of_week"]:
    if value == 0 or value == 1:
        result_1.append("Busiest days")
    elif value == 4 or value == 3:
        result_1.append ("Slowest days")
    else:
        result_1.append("Regularly busy")

In [30]:
result_1

['Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Slowest days',
 'Slowest days',
 'Regularly busy',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Regularly busy',
 'Regularly busy',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Regularly busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Regularly busy',
 'Busiest days',
 'Regularly busy',
 'Busiest days',
 'Busiest days',

# Q3.  Check values of new "busiest days" column for accuracy with observations in markdown format.

In [31]:
# Create new 'busiest_days' column in ords_prods_merge to view results
df_ords_prods_merge['busiest_days'] = result_1

In [32]:
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Regularly busy    12927461
Busiest days      11875462
Slowest days       7631289
Name: count, dtype: int64

In [35]:
# Check output
df_ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,first_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,busiest_days
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,False,5,0,both,Mid-range product,Regularly busy,Regularly busy
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,False,1,1,both,Mid-range product,Regularly busy,Regularly busy
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,False,20,0,both,Mid-range product,Busiest day,Busiest days
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,True,10,0,both,Mid-range product,Regularly busy,Slowest days
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,False,11,1,both,Mid-range product,Least busy,Slowest days


Q3 Answer.  Both "busiest-day" and "busiest-days" value counts have the same total count of 32,434,212 which means all rows have been group correctly.

# Q4.  Create new column "busiest_period_of_day" to show periods labeled "Most orders", "Average orders" and "Fewest orders."

In [36]:
# Checking value counts for each hour
df_ords_prods_merge ['order_hour_of_day'].value_counts()

order_hour_of_day
10    2764390
11    2738585
14    2691563
15    2664522
13    2663272
12    2620800
16    2537469
9     2456661
17    2089452
8     1719952
18    1637922
19    1259382
20     977017
7      891928
21     796362
22     634737
23     402612
6      290770
0      218942
1      115786
5       88057
2       69431
4       53283
3       51317
Name: count, dtype: int64

In [38]:
# Creating for-loop if statement and dividing hours_of_day into three sections "Most orders" = (10,11,14,15,13,12,16,9) "Fewest orders" = (23,6,0,1,5,2,4,3) "Average orders" = (17,8,18,19,20,7,21,22)

result_2 = []

for value in df_ords_prods_merge["order_hour_of_day"]:
    if value in [10, 11, 14, 15, 13, 12, 16, 9]:
        result_2.append("Most orders")
    elif value in [23, 6, 0, 1, 5, 2, 4, 3]:
        result_2.append("Fewest orders")
    else:
        result_2.append("Average orders")

In [39]:
result_2

['Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Fewest orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Fewest orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most ord

In [40]:
# Create column "busiest_period_of_day"
df_ords_prods_merge['busiest_period_of_day'] = result_2

# Q5. Print frequency of "busiest_period_of_day" column

In [41]:
# Print value counts in "busiest_period_of_day"column
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21137262
Average orders    10006752
Fewest orders      1290198
Name: count, dtype: int64

# Q7. Export dataframe as a pickle file to "Prepared Data" folder

In [42]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))